## <center>Tensorflow 实战 RNN</center>

### 1 单步RNN: RNNCell

RNNCell 它是 Tensorflow 中实现 RNN 的基本单元,每个 RNNCell 都有一个call 方法,使用方式:
```python
(output,next_state) = call(input,state)
```
借助图片来说可能更容易理解。假设我们有一个初始状态 $h_0$，还有输入 $x_1$，调用 call($x_1$, $h_0$)后就可以得到($output_1$, $h_1$)：
<br/><br/>
<center><img src="./img/9/1.jpg" width="600"/></center>
<br/><br/>
再调用一次 call($x_1$,$h_1$) 就可以得到 ($output_2,h_2$)
<br/><br/>
<center><img src="./img/9/2.jpg" width="600"/></center>

也就是说，每调用一次 RNNCell 的 call 方法，就相当于在时间上“推进了一步”，这就是 RNNCell 的基本功能。除了call方法外，对于RNNCell，还有两个类属性比较重要：
+ state_size
+ output_size

前者是隐层的大小，后者是输出的大小。比如我们通常是将一个 batch 送入模型计算，设输入数据的形状为(batch_size, input_size)，那么计算时得到的隐层状态就是(batch_size, state_size)，输出就是(batch_size, output_size)。

In [5]:
import tensorflow as tf
import numpy as np

cell = tf.contrib.rnn.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
output1, h1 = cell.__call__(inputs, h0) #调用call函数

print(h1.shape) # (32, 128)

128
(32, 128)


对于BasicLSTMCell，情况有些许不同，<font color='red'>因为LSTM可以看做有两个隐状态h和c</font>，对应的隐层就是一个Tuple，每个都是(batch_size, state_size)的形状：

In [6]:
import tensorflow as tf
import numpy as np
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = lstm_cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)  # shape=(32, 128)
print(h1.c)  # shape=(32, 128)

Tensor("basic_lstm_cell/Mul_2:0", shape=(32, 128), dtype=float32)
Tensor("basic_lstm_cell/Add_1:0", shape=(32, 128), dtype=float32)


### 2 多步RNN: tf.nn.dynamic_rnn

基础的 RNNCell 有一个很明显的问题：对于单个的 RNNCell，我们使用它的 call 函数进行运算时，只是在序列时间上前进了一步。比如使用 $x_1$、$h_0$ 得到 $h_1$，通过 $x_2$、$h_1$ 得到 $h_2$ 等。如果我们的序列长度为 10，就要调用 10 次 call 函数，比较麻烦。对此，TensorFlow 提供了一个 $tf.nn.dynamic\_rnn$ 函数，使用该函数就相当于调用了 $n$ 次 call 函数。即通过 {$h_0,x_1, x_2, …., x_n$} 直接得 {$h_1,h_2…,h_n$}。

具体来说，设我们输入数据的格式为 (batch_size, time_steps, input_size) ，其中 time_steps 表示序列本身的长度，如在 Char RNN 中，长度为 10 的句子对应的 time_steps 就等于10。最后的 input_size 就表示输入数据单个序列单个时间维度上固有的长度。另外我们已经定义好了一个 RNNCell，调用该 RNNCell 的 call 函数 time_steps 次

In [7]:
import tensorflow as tf
tf.reset_default_graph()
batch_size = 4 
# 通常输入数据的格式为 [timesteps,batch_size,input_data],其中input_data 表示当前RNN的输入. 
input = tf.random_normal(shape=[3, batch_size, 6], dtype=tf.float32)
# 这里使用最基本的 RNNCell ,10 表示一个RNNCell中有多少个隐藏单元
cell = tf.nn.rnn_cell.BasicRNNCell(10)
# 第一个RNNCell的输入为,输入数据和一个初始化的隐含状态,这个隐含状态通常初始化为0
#　初始化状态的大小就是 batch_size, 表明有 batch_size 个独立的状态
init_state = cell.zero_state(batch_size, dtype=tf.float32)
# output 表示每一个RNNCell 的输出,输出的形状为(3,4,10)
# final_state 表示最后一个 RNNCell 的输出状态,输出形状为(4,10)
output, final_state = tf.nn.dynamic_rnn(cell, input, initial_state=init_state, time_major=True)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(output).shape)
    print(sess.run(final_state))
    print(sess.run(final_state).shape)

(3, 4, 10)
[[ 0.15897425 -0.49791458 -0.4963269   0.5887911   0.8083712   0.20863965
  -0.02076925 -0.7549604   0.2806686   0.36615604]
 [ 0.78095293  0.938       0.60008585  0.5948752   0.25974703  0.039651
  -0.29459378  0.32053632  0.44302717  0.5970255 ]
 [-0.6674305   0.98681504  0.96141636 -0.6867633   0.50147516  0.31646675
  -0.64991957  0.92567086  0.43790862  0.65995336]
 [ 0.19428924 -0.7440126  -0.6446582  -0.8749729  -0.21022728 -0.7555016
   0.15203257  0.8128691  -0.9370999  -0.7597995 ]]
(4, 10)


### 3 堆叠RNNCell: MultiRNNCell

很多时候，单层 RNN 的能力有限，我们需要多层的 RNN。将 $x$ 输入第一层 RNN 的后得到隐层状态 $h$,这个隐层状态就相当于第二层 RNN 的输入，第二层 RNN 的隐层状态又相当于第三层 RNN 的输入，以此类推。在TensorFlow 中，可以使用 $tf.nn.rnn\_cell.MultiRNNCell$ 函数对 RNNCell 进行堆叠，相应的示例程序如下：

In [11]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
# 每调用一次这个函数就返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)
# 用tf.nn.rnn_cell MultiRNNCell创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(3)]) # 3层RNN
# 得到的cell实际也是RNNCell的子类
# 它的state_size是(128, 128, 128)
# (128, 128, 128)并不是128x128x128的意思
# 而是表示共有3个隐层状态，每个隐层状态的大小为128
print(cell.state_size) # (128, 128, 128)
# 使用对应的call函数
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = cell.call(inputs, h0)
print(h1) # tuple中含有3个32x128的向量

(128, 128, 128)
(<tf.Tensor 'cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)


### 4 Tensorflow 中的 cell 类

Tensorflow 中定义了 5 个常用的 cell 类,都可以在  $tf.contrib.rnn.XXXCell$  中找到, 具体定义如下:
+ BasicRNNCell:
```python
BasicRNNCell(num_units)
num_units: 每个 cell 包含的隐含单元个数
```
+ BasicLSTMCell:
```python
BasicLSTMCell(num_units,forget_bias=1.0,activation=tanh)
num_units: 每个 cell 包含的隐含单元个数
forget_bias: 添加到 forget 门的偏置
activation: 默认激活函数
```
+ LSTMCell:
```python
LSTMCell(num_units,use_peepholes=False,cell_cip=None,initializer=None,forget_bias=1.0)
num_units: 每个 cell 包含的隐含单元个数
cell_clip: 在输出前对 cell 状态按照给定值进行截断处理
initializer: 指定初始化函数
forget_bias: 添加到 forget 门的偏置
```
+ GRUCell:
```python
GRUCell(num_units)
num_units: 每个 cell 包含的隐含单元个数
```
+ MultiRNNCell:
```python
MultiRNNCell(cells,state_is_tuple=True)
cells: 一个 cell 列表,将列表中的 cell 一个个堆叠起来,如果使用 cells=[cell1,cell2],就是一共两层,数据经过 cell1 后还要经过 cell2
state_is_tuple: 如果是 True 则返回 n-tuple,将cell的输出值与状态组成一个 tuple .其中,输出值的结构为:c=[batch_size,num_units],输出状态的结构为 h=[batch_size,num_units]
```
<font color='red'>注意</font><br/>
在使用 MultiRNNCell 时,有些习惯的写法是 cells 参数直接使用 [cell]$\times$n 来代表创建 n 层的 cell, 这种写法如果不使用作用域隔离,则会报编译错误,或者使用一个外层循环将 cell 一个个 append 进去来解决命名冲突.

### 5 通过 cell 类构建 RNN

定义好 cell 类之后,还需要将他们链接起来构成 RNN 网络. Tensorflow 中主要有以下积累构建网络的模式,构建函数主要在两个包中,$tf.nn$ 和 $tf.contrib$ ,其中 $tf.contrib$ 表示由其他人提供的,而 $tf.nn$ 表示官方提供的稳定版:

#### (1) 静态 RNN 构建
Tensorflow 中提供了一个构建静态 RNN 的函数 $tf.nn.static\_rnn$, 定义如下:
```python
tf.nn.static_rnn(cell,inputs,initial_state=None,dtype=None,sequence_length=None,scope=None)
cell: 由 cell 类生成的对象
inputs: 输入数据,一般数据输入格式为(timesteps,batch_size,cell_data)
initial_state: 初始化 cell 状态
dtype: 期望输出和初始化 state 的类型
sequence_length: 每一个输入的序列长度,如果超过了 timesteps 就会截断,这也是静态 RNN 的一个缺点
返回值: 一个是结果,一个是 cell 状态,结果是一个list,输入是多少个时序,list 里面就会有多少个元素.
```

#### (2) 动态 RNN 构建
Tensorflow 中提供了一个构建动态 RNN 的函数 $tf.nn.dynamic\_rnn$, 定义如下:
```python
tf.nn.dynamic_rnn(cell,inputs,initial_state=None,dtype=None,sequence_length=None,scope=None)
cell: 由 cell 类生成的对象
inputs: 输入数据,一般数据输入格式为(batch_size,timesteps,cell_data)
initial_state: 初始化 cell 状态
dtype: 期望输出和初始化 state 的类型
sequence_length: 每一个输入的序列长度,如果超过了 timesteps 就会截断,这也是静态 RNN 的一个缺点
time_major: 默认值 False 时, input 的形状为 [batch_sise,timesteps,cell_data]. 如果是 True,形状为[timesteps,batch_size,cell_data]
返回值: 一个是结果,一个是 cell 状态,结果是以 [batch_size,timesteps,...] 形式的张量.
```
<font color="red">注意</font><br/>
动态 RNN 的输出,它是以 batch_size 优先的矩阵,因为我们需要取最后一个时序的输出,所以需要处理成一个以时间优先的形式.

#### (3) 双向 RNN 构建
双向 RNN 作为一个可以学习正,反向归来的循环神经网络,在 Tensorflow 中有3个函数可以使用:
+ tf.nn.bidirectional_dynamic_rnn
```python
bidirectional_dynamic_rnn(cell_fw,cell_bw,inputs,initial_states_fw,initial_states_bw)
cell_fw,cell_bw: 前向和反向的 Cell
inputs: 输入序列,一个张量输入,形状为[batch_size,timesteps,cell_data]
initial_states_fw,initial_states_bw: 前向和反向 cell 的初始状态
#
返回值: 是一个tuple(outputs,output_state_fw,output_state_bw),其中outputs也是一个
tuple(output_fw,output_bw),每个值是一个张量 [batch_size,timesteps,layers_output],
如果需要总的结果,可以将前后项的 layers_output 使用 tf.concat 连接起来
```

+ tf.contrib.rnn.stack_bidirectional_rnn
```python
创建一个静态的多层双向网络.输出作为下一层的输入,前后向的输入大小必须一致,两层之间是独立的,不能共享信息.
stack_bidirectional_rnn(cells_fw,cells_bw,inputs,initial_states_fw,initial_states_bw)
cells_fw,cells_bw: 前向和后向实例化之后的 cell 列表,正反向的 list 长度必须相等,输入必须相同
inputs: 输入为 [timesteps,batch_size,input_size] 形状的张量
initial_states_fw,initial_states_bw: 前向和后向的 cell 初始化状态
#
返回值: 是一个tuple(outputs,output_state_fw,output_state_bw),其中outputs为 
[timesteps,batch_size,layers_output] 形状的张量, layers_output 包含 tf.concat 之后的正反向的输出
```

+ tf.contrib.rnn.stack_bidirectional_dynamic_rnn
```python
创建一个动态的多层双向网络.输出作为下一层的输入,前后向的输入大小必须一致,两层之间是独立的,不能共享信息.
stack_bidirectional_dynamic_rnn(cells_fw,cells_bw,inputs,initial_states_fw,initial_states_bw)
cells_fw,cells_bw: 前向和后向实例化之后的 cell 列表,正反向的 list 长度必须相等,输入必须相同
inputs: 输入形状为 [batch_size,timesteps,input_size] 形状的张量,与静态的关键不同
initial_states_fw,initial_states_bw: 前向和后向的 cell 初始化状态
#
返回值: 是一个tuple(outputs,output_state_fw,output_state_bw),其中outputs为 
[batch_size,timesteps,layers_output] 形状的张量, layers_output 包含 tf.concat 之后的正反向的输出
```

<font color='red'>注意</font><br/>
在单层,双层,双向 RNN 函数的介绍中,都有动态和静态之分.静态的意思就是按照样本的时间序列个数 $n$ 展开,在图中创建 $n$ 个序列的 cell. 动态的意思是只创建样本中一个序列的 RNN,其他的序列数据都会通过循环来进入该 RNN 来运算.

通过静态生成的 RNN 网络,生成过程所需的时间比较长,网络所占有的内存会更多,导出的模型会更大.模型中会带有每个序列中间的状态信息,便于调试.并且所有样本的序列长度固定,过短需要填充,过长就会截断. 通过动态生成的 RNN 网络,所占的内存比较小,导出的模型较小.模型中只会有最后的状态,在使用过程中,支持不同的序列个数.

#### (4) 使用动态 RNN 处理变长序列
动态 RNN 还有个更高级的功能,就是可以处理变长序列,方法就是:<font color='red'>在准备样本的同时,将样本对应的长度也作为初始化参数,一起创建动态 RNN</font>

In [5]:
import tensorflow as tf
import numpy as np

In [7]:
tf.reset_default_graph()
x = np.random.randn(2,4,5)
x = x.astype(np.float32)
x[1,1:]=0
print(x)

[[[-0.07997223  0.5211962  -1.4125582   0.30114755  0.8558873 ]
  [-0.22222753  0.39622518 -0.5122942   0.63228947 -1.5071013 ]
  [-0.1456246  -0.19566543  1.4566741  -0.57916325  1.4118758 ]
  [ 0.46934992 -0.48371202  1.0825748   0.43605757 -2.632912  ]]

 [[-0.8880843   0.5106341  -1.2813077   0.59301436 -1.3673395 ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]]


In [8]:
seq_lengths = [4,1]
# 分别建立一个 lstm 和 gru 的 cell,比较两者的输出状态
cell = tf.contrib.rnn.BasicLSTMCell(num_units=3,state_is_tuple=True)
gru = tf.contrib.rnn.GRUCell(num_units=3)

# 如果没有 initial_state,必须指定 dtype
outputs,last_states = tf.nn.dynamic_rnn(cell,x,sequence_length=seq_lengths,dtype=tf.float32)
gru_outputs,gru_last_states = tf.nn.dynamic_rnn(gru,x,sequence_length=seq_lengths,dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
output,cell_state,gru_output,gru_state = sess.run([outputs,last_states,gru_outputs,gru_last_states])
print("full_seq:",output[0])
print("short_seq:",output[1])
print("lstm_cell_state: ",len(cell_state),"\n",cell_state[1])
print("gru_short_cell_state: ",gru_output[1])
print("gru_cell_state: ",len(gru_state),"\n",gru_state[1])

('full_seq:\n', array([[-0.14164376,  0.11737215, -0.1945096 ],
       [-0.07673898,  0.23758058, -0.10758322],
       [-0.00289119,  0.0114625 ,  0.06400157],
       [ 0.01706571,  0.04896582,  0.03375101]], dtype=float32))
('short_seq:\n', array([[-0.05792242,  0.28562078, -0.1128065 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]], dtype=float32))
('lstm_cell_state: ', 2, '\n', array([[ 0.01706571,  0.04896582,  0.03375101],
       [-0.05792242,  0.28562078, -0.1128065 ]], dtype=float32))
('gru_short_cell_state: \n', array([[ 0.02717191, -0.01924445,  0.29373556],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ]], dtype=float32))
('gru_cell_state: ', 2, '\n', array([ 0.02717191, -0.01924445,  0.29373556], dtype=float32))
